<a href="https://colab.research.google.com/github/Qcritt/cosc470s24/blob/main/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image
import numpy as np

def load_custom_images(directory, size=(32,32)):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory):
        # Construct the full path to the image
        path = os.path.join(directory, filename)
        # Open the image file
        image = Image.open(path)
        # Resize image (just in case some are not 32x32)
        image = image.resize(size)
        # Convert image to numpy array
        image_data = np.asarray(image)
        # Scale data to the range [-1, 1]
        image_data = (image_data - 127.5) / 127.5
        images.append(image_data)
    # Convert list to a numpy array
    images = np.array(images)
    return images


In [26]:
# Load your custom dataset
from PIL import Image
import os

def load_custom_images(directory):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory):
        # Construct the full path to the image
        path = os.path.join(directory, filename)
        # Check if the file is an image file
        if os.path.isfile(path) and any(path.endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            # Open the image and append it to the list
            image = Image.open(path)
            images.append(image)
    return images

# Load your custom dataset
from PIL import Image
import os
import numpy as np

def load_custom_images(directory):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory):
        # Construct the full path to the image
        path = os.path.join(directory, filename)
        # Check if the file is an image file
        if os.path.isfile(path) and any(path.endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            # Open the image, convert it to numpy array, and append it to the list
            image = Image.open(path)
            image_array = np.array(image)
            images.append(image_array)
    # Convert the list of images to a numpy array
    dataset = np.array(images)
    return dataset


import os

# Get the current working directory
current_directory = os.getcwd()
# Join the directory name with the current working directory
directory = os.path.join(current_directory, 'ResizedFrames')


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout
from keras.optimizers import Adam
import numpy as np

def define_discriminator(in_shape=(32, 32, 3)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))
    return model


In [15]:
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0001, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [16]:
from matplotlib import pyplot as plt
import numpy as np

def train_gan(g_model, d_model, gan_model, dataset, latent_dim, epochs=2, batch_size=128):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        for batch in range(dataset.shape[0] // batch_size):
            # randomly select real samples
            idx = np.random.randint(0, dataset.shape[0], half_batch)
            real_images = dataset[idx]
            real_y = np.ones((half_batch, 1))
            # generate fake images
            noise = np.random.randn(latent_dim * half_batch).reshape(half_batch, latent_dim)
            fake_images = g_model.predict(noise)
            fake_y = np.zeros((half_batch, 1))
            # update discriminator model
            d_model.train_on_batch(real_images, real_y)
            d_model.train_on_batch(fake_images, fake_y)
            # prepare points in latent space as input for the generator
            x_gan = np.random.randn(latent_dim * batch_size).reshape(batch_size, latent_dim)
            y_gan = np.ones((batch_size, 1))
            # update the generator via the discriminator's error
            gan_model.train_on_batch(x_gan, y_gan)

            # Plot a generated image after every batch
            if batch % 1 == 0:
                show_generated_image(g_model, latent_dim)

def show_generated_image(g_model, latent_dim):
    noise = np.random.randn(latent_dim).reshape(1, latent_dim)
    image = g_model.predict(noise)[0]
    image = (image + 1) / 2.0  # Rescale to [0, 1]
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [27]:
latent_dim = 100  # Dimensionality of the latent space

# Define models
discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)

# Load dataset (assuming this is already done)
dataset = load_custom_images('/content/')

# Train the GAN
train_gan(generator, discriminator, gan_model, dataset, latent_dim, epochs=100, batch_size=128)


It seems like the images quickly become black and I cant get them to show any light. I tried changing some parameter like learning rate and it didnt help.